# Spectrum Analysis on RFSoC
----
<div class="alert alert-box alert-info">
Please use Jupyter labs http://board_ip_address/lab for this notebook.
</div>

This notebook explores the frequency spectrum of the RF DCs.

## Aims
* Learn how to inspect and manipulate captured data from the RF ADCs.
* Apply a window to time domain data as a preprocessing step for spectrum analysis.
* Create a spectrum plot for visualization.

## Table of Contents

* [Hardware Setup](#hardware-setup)
* [Frequency Domain Analysis](#frequency-domain-analysis)
    * [Capture RF ADC Data](#capture-rf-adc-data)
    * [Time Domain Windowing](#time-domain-windowing)
    * [Convert to the Frequency Domain](#convert-to-the-frequency-domain)
    * [The Power Spectral Density](#the-power-spectral-density)
    * [Plotting the Spectrum](#plotting-the-spectrum)
* [Conclusion](#conclusion)

## References
* [Xilinx, Inc, "USP RF Data Converter: LogiCORE IP Product Guide", PG269, v2.4, November 2020](https://www.xilinx.com/support/documentation/ip_documentation/usp_rf_data_converter/v2_4/pg269-rf-data-converter.pdf)
* [Cooley, James W.; Tukey, John W. (1965). "An algorithm for the machine calculation of complex Fourier series". Mathematics of Computation. 19 (90): 297–301.](https://www.ams.org/journals/mcom/1965-19-090/S0025-5718-1965-0178586-1/home.html)

## Revision History
* v1.0 | 25/02/2022 | First notebook revision.

----

## Hardware Setup <a class="anchor" id="hardware-setup"></a>

The RFSoC4x2 board contains 4 RF ADC channels and 2 RF DAC channels. For this demonstration, we will be using 2 RF ADC channels and 2 RF DAC channels in loopback.

Make the loopback connection, using SMA cables, as shown below:

* Channel 3: DAC A to ADC A
* Channel 2: DAC B to ADC B

See the image below for a demonstration.

<img src='data/rfsoc4x2_loopback.jpg' align='left' style='left' height='75%' width='75%'/>

<div class="alert alert-heading alert-danger">
    <h4 class="alert-heading">Warning:</h4>

In this demo we are transmitting signals via cables.
This device can also transmit wireless signals. 
Keep in mind that unlicensed transmission of wireless signals
may be illegal in your location. 
Radio signals may also interfere with nearby devices,
such as pacemakers and emergency radio equipment. 
If you are unsure, please seek professional support.
</div>

----

## Frequency Domain Analysis <a class="anchor" id="frequency-domain-analysis"></a>

Frequency domain analysis is the process of inspecting properties
of the radio spectrum. There are several spectrum analysis algorithms and tools;
the most common is the FFT.

In this notebook, we will use the FFT on data
captured from the RF ADC frontend to analyze the spectral properties of the
received signal. The RF DAC will be used to provide stimulus through
loopback and the mixer frequency can be changed to generate different
input tones.

Start by downloading the base overlay.

In [1]:
from pynq.overlays.base import BaseOverlay

base = BaseOverlay('base.bit')

The RFSoC4x2 has a sophisticated clocking network, which can generate
low-jitter clocks for the RF DC Phase-Locked Loops (PLLs).

The base overlay has a simple method to initialize these clocks. Run the cell below to set
the LMK and LMX clocks to 245.76MHz and 491.52MHz, respectively.

In [2]:
base.init_rf_clks()

### Setting up the Transmitter
There are several interesting spectral properties to inspect.
We will set up each RF DAC channel with a carrier frequency first.
Then we can set the each channel to different frequencies and amplitudes
to make our example more interesting.

In [3]:
def set_transmitter_channel(channel, enable, gain, frequency):
    channel.control.enable = enable
    channel.control.gain = gain
    channel.dac_block.MixerSettings['Freq'] = frequency

set_transmitter_channel(base.radio.transmitter.channel[0], True, 0.8, 900)
set_transmitter_channel(base.radio.transmitter.channel[1], True, 0.1, 1500)

The above code cell has set the transmitter for each channel as follows:

* Transmitter Channel 0
    * Frequency: 900 MHz
    * Gain: 0.8
* Transmitter Channel 1
    * Frequency: 1500 MHz
    * Gain: 0.1
    
Lets look out for these frequencies when we finally reach the spectrum analysis section.

### Capture RF ADC Data <a class="anchor" id="capture-rf-adc-data"></a>

In order to perform spectral analysis, we need to capture data from the RF
ADC. We can capture data using the radio hierarchy block.

In [4]:
number_samples = 1229
sample_frequency = 2458e6
cdata = []
for i in range(0, len(base.radio.receiver.channel)):
    cdata.append(base.radio.receiver.channel[i].transfer(number_samples))

Great, let's now create time domain plots for each segment of data captured
from the RF ADC. 

In the cell below, we have created a simple helper function `plot_complex_time`
which can help us quickly create new complex time domain plots. 
We will use `plotly`, `numpy`, and `ipywidgets` to support our visualization.

In [5]:
import plotly.graph_objs as go
import numpy as np
import ipywidgets as ipw

def plot_complex_time(data, n=number_samples, fs=sample_frequency, 
                      title='Complex Time Plot'):
    plt_re_temp = (go.Scatter(x = np.arange(0, n/fs, 1/fs),
                              y = np.real(data), name='Real'))
    plt_im_temp = (go.Scatter(x = np.arange(0, n/fs, 1/fs),
                              y = np.imag(data), name='Imag'))
    return go.FigureWidget(data = [plt_re_temp, plt_im_temp],
                           layout = {'title': title, 
                                     'xaxis': {
                                         'title': 'Seconds (s)',
                                         'autorange' : True},
                                     'yaxis': {
                                         'title': 'Amplitude (V)'}})

tfigs = []
for i in range(2, len(base.radio.receiver.channel)):
    tfigs.append(plot_complex_time(
            data=cdata[i], 
            title=''.join(['Time Domain Plot of ADC Channel ', str(i)])))

ipw.VBox(tfigs)

    'data': [{'name': 'Real',
              'type': 'scatter',
              'ui…

Note that it is quite difficult to look at the plots and determine the signal's
oscillation frequency. This is where spectral analysis can prove to be very
useful. Let's first apply a simple window to our time domain signals.

### Time Domain Windowing <a class="anchor" id="time-domain-windowing"></a>

Windowing is a useful tool in spectral analysis of a time domain signal. 
This technique is used on the time domain signal before FFT. 

Why do we need windowing? The FFT assumes that the input signal is periodic.
To prevent unwanted frequency spurs and spectral leakage, the start of the
signal should also connect to the end of the signal.

In this example, we will use a simple Blackman
filter to illustrate windowing.

In [6]:
window = np.array(np.blackman(number_samples)[:])

The blackman window can be easily applied to the time domain data
through element-wise multiplication. This is why the window created in the
cell above, is the same size as our time domain data.

In [7]:
wdata = []
wfigs = []

for i in range(2, len(base.radio.receiver.channel)):
    wdata.append(cdata[i]*window)
    wfigs.append(plot_complex_time(
            data=wdata[i-2], 
            title=''.join(['Windowed Plot of ADC Channel ', str(i)])))

ipw.VBox(wfigs)

    'data': [{'name': 'Real',
              'type': 'scatter',
              'ui…

As you can see, the windowed data now starts and ends around an amplitude of 0.
This reduces spectral leakage and unwanted spurs when we later perform
spectral analysis.

### Convert to the Frequency Domain <a class="anchor" id="convert-to-the-frequency-domain"></a>

Converting to the frequency domain is very easy using the FFT. The `numpy`
package performs most of the hard work for us. Let's obtain the magnitude of
the frequency spectrum.

In [8]:
fdata = []
for i in range(0, len(wdata)):
    fdata.append(np.fft.fftshift(np.fft.fft(wdata[i])))

The FFT algorithm will always return the positive and
negative frequencies in reverse order. So we should swap these around
using `np.fft.fftshift` as given in the above cell. 

After shifting the frequency spectrum, we can plot to inspect the results.

Note that you may see some low amplitude peaks in the unconnected channels. These are spurious emissions (also known as spurs) caused by the RF ADC sampling process. You can read more about these spurs, as well as some other interesting RF-sampling characteristics, [here](https://www.xilinx.com/content/dam/xilinx/support/documentation/white_papers/wp509-rfsampling-data-converters.pdf).

In [9]:
def plot_complex_spectrum(data, N=number_samples, fs=sample_frequency, 
                          title='Complex Spectrum Plot', units='dBW', fc=0):
    plt_temp = (go.Scatter(x = np.arange(-fs/2, fs/2, fs/N) + fc,
                           y = data, name='Spectrum'))
    return go.FigureWidget(data = plt_temp,
                           layout = {'title': title, 
                                     'xaxis': {
                                         'title': 'Frequency (Hz)',
                                         'autorange': True},
                                     'yaxis': {
                                         'title': units}})

magfigs = []
for i in range(0, len(fdata)):
    magfigs.append(plot_complex_spectrum(
            data=abs(fdata[i]),
            title=''.join(['Frequency Magnitude Plot of ADC Channel ', str(i+2)]),
            units='|Y(f)|',
            fc=round(abs(base.radio.receiver.channel[i].adc_block.MixerSettings['Freq']))*1e6))

ipw.VBox(magfigs)

    'data': [{'name': 'Spectrum',
              'type': 'scatter',
             …

The code cell above introduced a new custom function `plot_complex_spectrum`
to help us plot complex frequency domain plots. This function has an optional
argument `fc` to shift the plot's x-axis around the RF ADC block center frequency.

Inspecting the plots, you will see a peak around 900 MHz on channel 2, 
and 1500 MHz on channel 3. Notice the power of the signal for each channel.

### The Power Spectral Density <a class="anchor" id="the-power-spectral-density"></a>
A common way engineers typically inspect the frequency spectrum is by calculating the Power Spectral Density (PSD).

$PSD[n] = 10 \log_{10}(\frac{{|y[n]|^{2}}}{f_{s} \sum^{N_{w}}_{n=0}w^{2}[n]})$

where $w$ is the window vector, $f_s$ is the sample frequency, $N_{w}$ is the length of the window, and $y_{n}$ is the output of the FFT derived above.

Run the code cell below to calculate the PSD.

In [10]:
def freq_to_psd(data, N=number_samples, fs=sample_frequency, 
                window=np.array(np.ones(number_samples)[:])):
    psd = (abs(data)**2)/(fs*np.sum(window**2))
    return 10*np.where(psd > 0, np.log10(psd), 0)

psd = []
for i in range(0, len(fdata)):
    psd.append(freq_to_psd(data=fdata[i], window=window))

### Plotting the Spectrum <a class="anchor" id="plotting-the-spectrum"></a>

Finally, we can do the frequency plots. We will plot the PSD similar to
before and inspect the spectrum for each channel.

In [11]:
psdfigs = []
for i in range(0, len(psd)):
    psdfigs.append(plot_complex_spectrum(
            data=psd[i],
            title=''.join(['Power Spectral Density Plot of ADC Channel ',
                           str(i+2)]),
            units='Amplitude (dB)',
            fc=round(abs(base.radio.receiver.channel[
                        i].adc_block.MixerSettings['Freq']))*1e6))

ipw.VBox(psdfigs)

    'data': [{'name': 'Spectrum',
              'type': 'scatter',
             …

You will be able to see the peaks of each tone that was generated from the RF
DAC for each channel. Channel 2 was 900 MHz and channel 3 was 1500 MHz. 

Don't forget to investigate the power of the signal too. Channel 2 has better
signal strength as we gave it a larger gain using the amplitude controller.

## Conclusion <a class="anchor" id="conclusion"></a>

This notebook has presented RFSoC spectrum analysis. You have learned how
to inspect and manipulate captured data from the RF ADCs, applied a window
to time domain data, and created a spectrum plot for the PSD. 

The next notebook investigates a spectrum sweep application using the
above visualization and analysis principles.

[⬅️ RF Data Converter Introduction](01_rf_dataconverter_introduction.ipynb) | | [RF Spectrum Sweep ➡️](03_rf_spectrum_sweep.ipynb)

---
Copyright (C) 2022 Xilinx, Inc

SPDX-License-Identifier: BSD-3-Clause

----

----